# Agentics Mini Tutorial

Agentics provides the implementation of **AG**, a powerful datatype that connects
LLMs to Pydantic objects and enables **logical transduction**.

---

## Installation

```bash
!uv pip install agentics-py

In [1]:
! uv pip install agentics-py


import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    
    CURRENT_PATH=os.getcwd()
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 16ms
In Colab: False


## Use Agentics as Lists

Agentics objects (`AG`) can be used similarly to Python lists, allowing you to store and manage collections of states. You can append new elements using the `.append()` method, and access all states via the `.states` attribute.

For example, after creating an empty `AG` object, you can add elements:

In [2]:
from agentics import AG
my_first_agentics = AG()

print("The agentics is empty :", len(my_first_agentics))

 ## Add elements to the list
my_first_agentics.append("Alfio")
## internally, agentics stores the elements in the attribute states
my_first_agentics.states += ["Naweed" , "Junkyuu"] 

print("The agentics now has more instances :",len(my_first_agentics))

try:
    print("this triggers an error")
    my_first_agentics = my_first_agentics + my_first_agentics
except:
    my_first_agentics.states= my_first_agentics.states + my_first_agentics.states
    print("This is the right way to concetenate two agentics. Be careful, the states should be instances of the same atype")
    print(my_first_agentics.pretty_print())

print("Iterating over agentics:") 
for state in  my_first_agentics:
    print(state)

print("Be careful, the AG itself is not a list :" , my_first_agentics) 



2025-09-29 06:38:45.144 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


The agentics is empty : 0
The agentics now has more instances : 3
this triggers an error
This is the right way to concetenate two agentics. Be careful, the states should be instances of the same atype
Atype : None
Alfio
...

Naweed
...

Junkyuu
...

Alfio
...

Naweed
...

Junkyuu
...


Atype : None
Alfio
...

Naweed
...

Junkyuu
...

Alfio
...

Naweed
...

Junkyuu
...


Iterating over agentics:
Alfio
Naweed
Junkyuu
Alfio
Naweed
Junkyuu
Be careful, the AG itself is not a list : atype=None crew_prompt_params={'role': 'Task Executor', 'goal': 'You execute tasks', 'backstory': 'You are always faithful and provide only fact based answers.', 'expected_output': 'Described by Pydantic Type'} instructions='Generate an object of the specified type from the following input.' llm=<crewai.llm.LLM object at 0x13fb080e0> max_iter=3 prompt_template=None reasoning=None skip_intentional_definition=False states=['Alfio', 'Naweed', 'Junkyuu', 'Alfio', 'Naweed', 'Junkyuu'] tools=None transduce_fields=None 

## Atypes

Agentics supports **typed AGs** using Pydantic models, enabling you to enforce schema validation and structure on the states stored in an AG. This is useful when you want all elements in your AG to follow a specific format or contain certain fields.

To define a typed AG:

1. **Create a Pydantic model** that describes the schema for your states.
2. **Instantiate an AG** with the `atype` parameter set to your Pydantic model.
3. **Add instances** of your model to the AG. Only objects matching the schema will be accepted.

This approach ensures data consistency and allows you to leverage Pydantic's validation features within Agentics workflows.

For example, you can define a `Movie` type and create an AG that only accepts `Movie` instances as its states. See the next cell for a practical demonstration.

In [3]:
from pydantic import BaseModel
from typing import Optional

# Define the Movie Pydantic model for use with Agentics AG
class Movie(BaseModel):
    movie_name: Optional[str] = None
    genre: Optional[str] = None
    description: Optional[str] = None


movies = AG(atype=Movie)
movies.append(Movie(movie_name="La dolce vita"))
print(movies.pretty_print())


2025-09-29 06:38:45.152 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null




## Extending and Merging AGs
AGs can evolve by adding new fields or combining with other AGs to form richer schemas.

### Add attributes
Use `.add_attribute()` to dynamically extend the schema of an AG.  
This operation mutates the AG in place.

In [4]:
movies = AG(atype=Movie)
movies.append(Movie(movie_name="La dolce vita"))
movies.pretty_print()

print("adding a new attribute to the type and rebinding the object")
movies.add_attribute("email", 
                     description="Write an email to tell a fried about this movie")

print(movies.pretty_print())
print("Note that the AG changed")


2025-09-29 06:38:45.167 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


adding a new attribute to the type and rebinding the object
Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Note that the AG changed


### Subtypes
You can project an AG onto a subset of its fields, e.g. `movies("title", "genre")`.  
This creates a new AG without modifying the original.

In [5]:
movies_subtype = movies("movie_name", "genre")
print("This is a subtype")
movies_subtype.pretty_print()

print("This is the original type.\nNote that the AG didn't change after subtype")
print(movies.pretty_print())
print("Note that the AG didn't change after subtyping it")

This is a subtype
Atype : <class 'agentics.core.agentics.movie_name_genre'>
movie_name: La dolce vita
genre: null


This is the original type.
Note that the AG didn't change after subtype
Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Atype : <class '__main__.Movie'>
movie_name: La dolce vita
genre: null
description: null


Note that the AG didn't change after subtyping it


### Merge AGs
You can merge AGs of different types (e.g., `Movie` with `Director`), combining their states into a new AG with a union of fields.  
On field conflicts, values from the right-hand AG’s states take precedence.

In [6]:
# Define a new Pydantic type for directors
class Director(BaseModel):
    director_name: Optional[str] = None

# Merge movies (Movie type) with directors (Director type)
# The result AG will have fields from both Movie and Director
prod_movies = movies.merge(
    AG(atype=Director, states=[Director(director_name="Fellini")])
)

# Add another movie to the original AG
movies.append(Movie(movie_name="Superman"))

print("Merging AGs will combine states:")

# Merge with one director state; the single director is aligned with each movie
movies.merge(
    AG(atype=Director, states=[Director(director_name="Fellini")])
).pretty_print()

# Merge with two director states; directors are aligned by index with movies
# If the AGs are different lengths, extra states are still included
print(movies.merge(
    AG(atype=Director, states=[
        Director(director_name="Fellini"),
        Director(director_name="Donner")
    ])
).pretty_print())

2025-09-29 06:38:45.183 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.184 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.184 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.185 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.186 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.187 | DEBUG    | agentics.core.llm_connection

Merging AGs will combine states:
Atype : <class 'agentics.core.agentics.Movie__merge__Director'>
movie_name: La dolce vita
genre: null
description: null
director_name: Fellini

movie_name: Superman
genre: null
description: null
director_name: null


Atype : <class 'agentics.core.agentics.Movie__merge__Director'>
movie_name: La dolce vita
genre: null
description: null
director_name: Fellini

movie_name: Superman
genre: null
description: null
director_name: Donner


Atype : <class 'agentics.core.agentics.Movie__merge__Director'>
movie_name: La dolce vita
genre: null
description: null
director_name: Fellini

movie_name: Superman
genre: null
description: null
director_name: Donner




## Import Agentics for Json and CSV 

Agentics AG objects can be easily imported from and exported to CSV and JSONL formats. This enables seamless integration with tabular and structured data workflows.

- **CSV Import/Export:**  
    Use `AG.from_csv("path/to/file.csv")` to create an AG from a CSV file. The schema (`atype`) can be inferred automatically or provided explicitly.
- **JSONL Import/Export:**  
    Use `AG.to_jsonl("path/to/file.jsonl")` to export, and `AG.from_jsonl("path/to/file.jsonl")` to import AG objects in JSON Lines format.

This functionality allows you to move data between Agentics and other tools with minimal effort.

In [ ]:
# Create a new AG object from the provided csv file
movies = AG.from_csv(base / "data/movies.csv", max_rows=3)
movies.pretty_print()

# Note that the atype has been automatically inffered
print("Imported Type", movies.atype)

# Reloading same file by providing atype
movies = AG.from_csv(base /"data/movies.csv", atype=Movie)

# Note that just a subset of the attributes have been imported
print("Provided Type" , movies.atype)

# agentics can be exported and imported from jsonl objects
movies.to_jsonl(base /"data/movies.jsonl")
movies= AG.from_jsonl(base / "data/movies.jsonl")

# note this type is different from what imported from csv
print("Imported atype from jsonl: ", movies.atype)

2025-09-29 06:38:45.194 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.194 | DEBUG    | agentics.core.agentics:from_csv:303 - Importing Agentics of type Movie from CSV /Users/gliozzo/Code/agentics911/agentics/tutorials/data/movies.csv
2025-09-29 06:38:45.196 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.196 | DEBUG    | agentics.core.agentics:to_jsonl:409 - Exporting 100 states or atype <class '__main__.Movie'> to /Users/gliozzo/Code/agentics911/agentics/tutorials/data/movies.jsonl
2025-09-29 06:38:45.200 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class 'agentics.core.atype.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Drama, Crime
description: In the

## Logical Transduction

Once an AG is initialized with an atype, Agentics can **transduce** any string of text and/or pydantic object into that type.  If a list of strings is provided, they are processed asynchronously.

## Untyped transduction

If no target atype is provided, transduction works as a regular llm call, where the input text or pydantic object is given to the LLM and the output is the LLM response. In this use case, agentics provides an off the shelp **async scale-out framework for LLM calls**. 

Note that no AType is specified, the output of transduction is alist of strings. So it is not recommended to use this notation for transduction algebra. In addition, Unconstrained trnasduction tends to me less efficient as it requires the LLM to guess the type of output required, often resulting in verbose and unecessary information . 

In [8]:
from agentics import AG
import time


questions= [
    "What are the benefits of using Agentic AI for data workflows?",
    "Will AI improve working conditions for the middle class?",
    "How can Agentic AI enhance decision-making in finance?",
    "What risks should companies consider when adopting AI agents?",
    "Can AG objects integrate with existing data pipelines?",
    "Who won the latest FIFA worldcup"
]
start= time.time()
answers = await (AG() << questions)
end= time.time()

for answer in answers: print(answer[:100])
print(f"Uncostrained transduction done in {end-start} seconds")



2025-09-29 06:38:45.208 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:38:45.209 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

## Why Agentic AI Is a Game‑Changer for Data Workflows

| Benefit | What It Means for Your Data Pipe
**Short answer:** AI has the potential to improve many aspects of middle‑class work—productivity, sa
## Agentic AI — What It Is

**Agentic AI** refers to autonomous, goal‑driven software agents that ca
## Risks Companies Should Consider When Deploying AI Agents  

Below is a structured overview of the
**Short answer:** Yes—AG objects are designed to be pluggable, so they can be woven into most modern
The most recent FIFA World Cup that has been completed is the **2022 men’s tournament** held in Qata
Uncostrained transduction done in 33.36398386955261 seconds


### Transduction into Atype

You can define a target schema with Pydantic (e.g., `Answer`) and transduce text into it.  
The LLM output is parsed and validated into the fields `answer`, `justification`, and `confidence`.  
Note that the output is more clean and organized, and the time required to execute the transduction is one order of magnitude lower. 

In [ ]:
# Define a Pydantic model for a structured answer
class Answer(BaseModel):
    # The main response text
    answer: Optional[str] = None
    # An explanation or reasoning behind the answer
    justification: Optional[str] = None
    # A numeric confidence score (e.g. from 0.0 to 1.0)
    confidence: Optional[float] = None

# Transduce a natural language question into the structured Answer schema
start= time.time()
answers = await (AG(atype=Answer) << questions)
end= time.time()
print(f"Typed transduction done in {end-start} seconds")
print(answers.pretty_print())

2025-09-29 06:39:18.592 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Typed transduction done in 2.990898847579956 seconds
Atype : <class '__main__.Answer'>
answer: "Agentic AI can boost financial decision\u2011making by autonomously gathering\
  \ and integrating real\u2011time market, economic, and alternative data; continuously\
  \ analyzing this information with advanced models; running rapid scenario simulations\
  \ to forecast outcomes under different assumptions; providing personalized, goal\u2011\
  aligned investment or risk\u2011management recommendations; monitoring compliance\
  \ and regulatory changes; and learning from outcomes to refine its strategies over\
  \ time. This end\u2011to\u2011end, self\u2011directed capability enables faster,\
  \ more data\u2011driven, and adaptive decisions than traditional static models."
justification: "Agentic AI combines goal\u2011oriented autonomy, tool use, and continual\
  \ learning, allowing it to act as an independent analyst that can fetch data, run\
  \ complex models, and adjust its behavior 

### Transduction Between AGs

You can control transduction more precisely by converting **from one AG to another**:
- The **source AG** provides the input states (rendered via the prompt).
- The **target AG** defines the output schema and validation.
- Agentics renders each source state → sends it to the LLM → parses into the target type.

This pattern is ideal when you want consistent, structured outputs from heterogeneous inputs while keeping prompts and schema separate.

### Transduction Between AGs  
Here we convert product reviews (`ProductReview`) into sentiment summaries (`SentimentSummary`).  
The source AG provides the reviews, and the target AG enforces structured outputs (positive/neutral/negative with a reason).  

In [10]:
from typing import Optional, Literal
from pydantic import BaseModel
from agentics import AG

# Source schema: product reviews
class ProductReview(BaseModel):
    reviewer: Optional[str] = None
    text: Optional[str] = None
    stars: Optional[int] = None

# Target schema: summarized sentiment
class SentimentSummary(BaseModel):
    customer_sentiment: Optional[Literal["positive", "neutral", "negative"]] = None
    reason: Optional[str] = None

# Example reviews
reviews = [
    ProductReview(reviewer="Alice", text="Excellent quality and fast delivery!", stars=5),
    ProductReview(reviewer="Bob", text="Okay, but packaging was damaged", stars=3),
    ProductReview(reviewer="Carol", text="Terrible, broke after one use", stars=1),
]

# Create source and target AGs
source = AG(atype=ProductReview, states=reviews)
target = AG(atype=SentimentSummary)

# Transduce reviews into sentiment summaries
sentiments = await (target << source)
print(sentiments.pretty_print())

2025-09-29 06:39:21.593 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:39:21.594 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.SentimentSummary'>
customer_sentiment: positive
reason: Excellent quality and fast delivery

customer_sentiment: neutral
reason: Packaging was damaged

customer_sentiment: negative
reason: broke after one use


Atype : <class '__main__.SentimentSummary'>
customer_sentiment: positive
reason: Excellent quality and fast delivery

customer_sentiment: neutral
reason: Packaging was damaged

customer_sentiment: negative
reason: broke after one use




### Self-Transduction  

You can transduce within the same AG type by selecting different subsets of fields.  
This is useful for projecting, comparing, or enriching dataframes and state graphs without changing the original AG.  

In [11]:
movies = AG.from_csv(base / "data/movies.csv",atype=Movie, max_rows=20)
self_transductions = await movies.self_transduction(["movie_name","description"],["genre"])
print(self_transductions.pretty_print())

2025-09-29 06:39:23.207 | DEBUG    | agentics.core.agentics:from_csv:303 - Importing Agentics of type Movie from CSV /Users/gliozzo/Code/agentics911/agentics/tutorials/data/movies.csv
2025-09-29 06:39:23.208 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Atype : <class '__main__.Movie'>
movie_name: The Shawshank Redemption
genre: Drama, Romance, Comedy
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Romance, Comedy
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Drama, Crime
description: In the continuing saga of the Corl

### Customizing Transduction  

You can fine-tune how logical transduction works by configuring:  

- **LLMs** – choose the underlying language model to run the transduction.  
- **Instructions** – add task-specific guidance for the LLM.  
- **Prompt Templates** – control how inputs are rendered into prompts.  
- **Few-Shot Examples** – provide examples to steer the model’s behavior.  
- **Verbose Options** – enable detailed logging and debug outputs.  

#### Task instructions
The example below illustrate how to provide a llm and task specific instructions to transduction

In [12]:
questions_answering_ag=AG(atype=Answer,
                          llm=AG.get_llm_provider("watsonx"),
                          instructions= "Answer in italian")

print((await (questions_answering_ag << questions)).pretty_print())


Output()

Atype : <class '__main__.Answer'>
answer: "L'Agentic AI pu\xF2 migliorare il processo decisionale in ambito finanziario\
  \ fornendo analisi predittive pi\xF9 accurate, automatizzando la valutazione di\
  \ scenari complessi e adattando le strategie in tempo reale. Grazie alla capacit\xE0\
  \ di agire autonomamente, questi sistemi possono monitorare continuamente i mercati,\
  \ identificare opportunit\xE0 di investimento, gestire i rischi e ottimizzare i\
  \ portafogli, riducendo i tempi di risposta e gli errori umani."
justification: "Le tecnologie Agentic AI combinano apprendimento automatico avanzato,\
  \ ragionamento basato su obiettivi e capacit\xE0 di esecuzione autonoma. Questo\
  \ permette di elaborare grandi volumi di dati finanziari (prezzi, notizie, indicatori\
  \ macroeconomici) in modo pi\xF9 rapido rispetto agli analisti umani, di simulare\
  \ molteplici scenari di mercato e di adattare le decisioni in base a feedback continui.\
  \ Inoltre, l'automazione delle op

#### Prompt templates

Prompt templates enable greater customization of your transductions by providing a langchain style abstraction to render pydantic objects into input prompts for the agent. 



In [13]:
questions_answering_ag=AG(atype=Answer)

dow_jones_data=AG.from_csv("data/dow_jones.csv")
dow_jones_data =dow_jones_data.get_random_sample(0.002)
dow_jones_data.prompt_template="what happened to the financial markets in {date}?"
answers = await (questions_answering_ag << dow_jones_data)
print(answers.pretty_print())



2025-09-29 06:39:32.206 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'
2025-09-29 06:39:32.222 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

2025-09-29 06:39:38.238 | DEBUG    | agentics.core.async_executor:execute:65 - retrying 1 state(s), attempt 1


Atype : <class '__main__.Answer'>
answer: "On May 27, 2009, U.S. equity markets continued their post\u2011crisis recovery.\
  \ The Dow Jones Industrial Average closed at 8,500.31, up about 0.5% on the day;\
  \ the S&P 500 finished at 822.30, up roughly 0.6%; and the Nasdaq Composite rose\
  \ about 0.7% to near 1,800. The modest gains were driven by growing optimism about\
  \ fiscal stimulus, improving economic indicators, and the easing of the credit crunch."
justification: Historical market data for that date shows all three major U.S. indices
  posting gains, reflecting investor confidence as the recession eased and stimulus
  measures began to show effect.
confidence: 0.78

answer: On February 6, 2012, global equity markets posted modest gains. In the United
  States, the Dow Jones Industrial Average rose about 0.5% and the S&P 500 increased
  roughly 0.6%, while European markets also edged higher. The rally was driven by
  positive economic data, easing concerns over the Eurozo

## Tool Usage  

Agentics integrates seamlessly with the **MCP ecosystem**, allowing AGs to call external tools during transduction.  In addition to that, they also allows the use of CrewAI tools, as the underlying transduction framework is currently based on crewAI agents. 

This makes it easy to fetch, process, or enrich data dynamically while keeping results structured. 

In the following example we illustrate the use of Duck Duck Go search to improve the information gathering of historical market data. 

In [ ]:
from crewai.tools import tool
from ddgs import DDGS


## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("web_search")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))


questions_answering_ag.verbose_agent = True
questions_answering_ag.tools=[web_search]
dow_jones_data.filter_states(end=1)
answers = await (questions_answering_ag << dow_jones_data)
print(answers.pretty_print())

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: df446bc0-4f93-4310-89aa-165d30fc577e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│  Generate an object of the specified type from the following input. SOURCE:                                     │
│  what happened to the financial markets in 2009-05-27?                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

 Received None or empty response from LLM call.

 An unknown error occurred. Please check the details below.

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  We need to format as per spec: Thought, Action, Action Input, Observation.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 20830955-1fd5-4cdb-821b-6ff803682f21                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: df446bc0-4f93-4310-89aa-165d30fc577e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: We need to format as per spec: Thought, Action, Action Input, Observation.                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Atype : <class '__main__.Answer'>
answer: Thought, Action, Action Input, Observation
justification: The user requested the ReAct style format (Thought, Action, Action
  Input, Observation). According to the system instruction, this request is provided
  as the answer field in the required JSON structure.
confidence: 0.95


Atype : <class '__main__.Answer'>
answer: Thought, Action, Action Input, Observation
justification: The user requested the ReAct style format (Thought, Action, Action
  Input, Observation). According to the system instruction, this request is provided
  as the answer field in the required JSON structure.
confidence: 0.95


